In [2]:
import datetime
import sys
import os

import IPython
import IPython.display
import numpy as np
import pandas as pd
import tensorflow as tf
import kerastuner as kt
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model
from cond_rnn import ConditionalRNN
from sklearn.preprocessing import MinMaxScaler
from pmdarima.arima import auto_arima

import src.preprocessing
from src.functions import load_data, TimeSeriesTensor, create_evaluation_df, series_to_supervised, plot_train_history, validation

np.set_printoptions(threshold=sys.maxsize)

In [24]:
GEP1 = pd.read_csv('../data/GEP/Consumption_1H.csv', index_col=0, header=0, names=['value'])
GEP4 = pd.read_csv('../data/GEP/B4_Consumption_1H.csv', index_col=0, header=0, names=['value'])
datasets = [GEP1, GEP4]
names = ['GEP1', 'GEP4']

In [13]:
class Baseline(tf.keras.Model):
    def __init__(self, label_index=None):
        super().__init__()
        self.label_index = label_index

    def call(self, inputs):
        if self.label_index is None:
            return inputs
        result = inputs[:, :, self.label_index]
        return result[:, :, tf.newaxis]

In [14]:
metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(28))
from progressbar import ProgressBar
pbar = ProgressBar()
dX_train = []
dX_test = []
dX_scaler = []
for i,df in enumerate(datasets):
    train_inputs, test_inputs, y_scaler = MIMO_fulldata_preparation(df, n_test=8760, T=72, HORIZON=72)
    baseline = Baseline(label_index = 0)
    baseline.compile(loss=tf.losses.MeanSquaredError(), metrics=[tf.metrics.MeanSquaredError()])
    predictions_B = baseline.predict(test_inputs['X'])
    eval_df_B = create_evaluation_df(predictions_B.reshape(-1,72), test_inputs, 72, y_scaler)
    mape = validation(eval_df_B['prediction'], eval_df_B['actual'], 'MAPE')
    mae = validation(eval_df_B['prediction'], eval_df_B['actual'], 'MAE')
    rmse = validation(eval_df_B['prediction'], eval_df_B['actual'], 'RMSE')
    metrics.loc[i] = pd.Series({'mae':mae, 'mape':mape, 'rmse':rmse, 'B': i})
metrics.to_csv('./results/GEP/3days/baseline.csv')

In [23]:
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 72

class MyTuner(kerastuner.tuners.BayesianOptimization):
    def run_trial(self, trial, *args, **kwargs):
        # You can add additional HyperParameters for preprocessing and custom training loops
        # via overriding `run_trial`
        kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 1500, 2000, step=100)
        super(MyTuner, self).run_trial(trial, *args, **kwargs)# Uses same arguments as the BayesianOptimization Tuner.
        
class MyHyperModel(HyperModel):
    def __init__(self,ins,outs,name):
        self.ins  = ins
        self.outs = outs
        self.name = name

    def build(self,hp):
        inputs = keras.Input(shape=(self.ins,))
        l = hp.Int('layers',1,2)
        drop = hp.Float('dropout',min_value=0.1,max_value=0.3,step=0.1)
        n = hp.Int('neurons',32,256,step=32)
        if l==1:
            tf.keras.models.Sequential([
                tf.keras.layers.LSTM(32, input_shape=(HORIZON, 15)),
                tf.keras.layers.Dense(HORIZON)
            ])
        elif l==2:
            x1 = layers.Dense(n,kernel_initializer='normal',activation='relu')(inputs)
            x2 = layers.Dropout(drop)(x1)
            x3 = layers.Dense(n,kernel_initializer='normal',activation='relu')(x2)
            x4 = layers.Dropout(drop)(x3)
            outputs = layers.Dense(self.outs,kernel_initializer='normal',activation='linear')(x4)
        # Compile model
        model.compile(loss='mse', optimizer='adam',metrics=['mse',mape])
        return model

tuner = MyTuner(...)
# Don't pass epochs or batch_size here, let the Tuner tune them.
tuner.search(...)


FULL_LSTMIMO = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, input_shape=(HORIZON, 15)),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(HORIZON)
])


metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(28))
from progressbar import ProgressBar
pbar = ProgressBar()
dX_train = []
dT_train = []
dX_test = []
dX_scaler = []
for i,df in enumerate(datasets):
    train_inputs, test_inputs, y_scaler = MIMO_fulldata_preparation(df, n_test=4380, T=72, HORIZON=72)
    dX_train.append(train_inputs['X'])
    dT_train.append(train_inputs['target'])
    dX_test.append(test_inputs)
    dX_scaler.append(y_scaler)
global_inputs_X = tf.concat(dX_train, 0)
global_inputs_T = tf.concat(dT_train, 0)
#test_inputs = pd.concat(dn_test, axis=1)

# full data LSTM MIMO compilation and fit
FULL_LSTMIMO.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
        
history = FULL_LSTMIMO.fit(global_inputs_X, global_inputs_T, batch_size=32, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)
for i,df in enumerate(datasets):
    FD_predictions = FULL_LSTMIMO.predict(dX_test[i-1]['X'])
    FD_eval_df = create_evaluation_df(FD_predictions, dX_test[i-1], HORIZON, dX_scaler[i-1])
    mae = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAE')
    mape = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAPE')
    rmse = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'RMSE')
    #print('rmse {}'.format(rmse))
    metrics.loc[i] = pd.Series({'mae':mae, 'mape':mape, 'rmse':rmse, 'B': names[i]})
metrics.to_csv('./results/GEP/global/3days/fulldata_LSTM.csv')

Epoch 1/100
1356/1356 [==============================] - 42s 31ms/step - loss: 0.0235 - mean_squared_error: 0.0235 - val_loss: 0.0211 - val_mean_squared_error: 0.0211
Epoch 2/100
1356/1356 [==============================] - 42s 31ms/step - loss: 0.0157 - mean_squared_error: 0.0157 - val_loss: 0.0210 - val_mean_squared_error: 0.0210
Epoch 3/100
1356/1356 [==============================] - 42s 31ms/step - loss: 0.0144 - mean_squared_error: 0.0144 - val_loss: 0.0202 - val_mean_squared_error: 0.0202
Epoch 4/100
1356/1356 [==============================] - 42s 31ms/step - loss: 0.0136 - mean_squared_error: 0.0136 - val_loss: 0.0201 - val_mean_squared_error: 0.0201
Epoch 5/100
1356/1356 [==============================] - 42s 31ms/step - loss: 0.0130 - mean_squared_error: 0.0130 - val_loss: 0.0198 - val_mean_squared_error: 0.0198
Epoch 6/100
1356/1356 [==============================] - 42s 31ms/step - loss: 0.0125 - mean_squared_error: 0.0125 - val_loss: 0.0194 - val_mean_squared_error: 0.019

In [19]:
def train_test_split(df, n_test):
    if len(df) < 8760:
        n_test = round(len(df) * 0.2)
    test_df = df.copy()[-(n_test+71):]
    train_df = df.copy()[:-(len(test_df)-71)]
    return train_df, test_df

In [20]:
def ConditionalRNN_data_preparation(df, n_test=4380, T=72, HORIZON=72):
    df = src.preprocessing.preprocess(df, 'Belgium')
    c2 = series_to_supervised(df)
    c1 = df.iloc[:,1:9]
    c1 = c1.loc[c2.index]
    short_df = df.iloc[:,[0,1,-4,-3,-2,-1]].copy()
    #check how to formulate more correctly
    short_df = short_df[146:]
    train_df, test_df = train_test_split(short_df, n_test)
    train_c1, test_c1 = train_test_split(c1, n_test)
    train_c2, test_c2 = train_test_split(c2, n_test)
    y_scaler = MinMaxScaler()
    y_scaler.fit(train_df[['value']])
    train_c2[['value(t-24)']] = y_scaler.fit_transform(train_c2[['value(t-24)']])
    train_c2[['value(t-168)']] = y_scaler.fit_transform(train_c2[['value(t-168)']])
    X_scaler = MinMaxScaler()
    train_df[train_df.columns] = X_scaler.fit_transform(train_df)
    test_df[train_df.columns] = X_scaler.fit_transform(test_df)
    tensor_structure = {'X':(range(-T+1, 1), ['value','fractional hour_sin','fractional hour_cos','day of year_sin','day of year_cos'])}
    train_inputs = TimeSeriesTensor(train_df, 'value', HORIZON, tensor_structure)
    test_inputs = TimeSeriesTensor(test_df, 'value', HORIZON, tensor_structure)
    train_c1 = c1.reindex(train_inputs.dataframe.index)
    train_c2 = c2.reindex(train_inputs.dataframe.index)
    test_c1 = c1.reindex(test_inputs.dataframe.index)
    test_c2 = c2.reindex(test_inputs.dataframe.index)
    return train_inputs, test_inputs, train_c1, test_c1, train_c2, test_c2, y_scaler

In [21]:
def MIMO_fulldata_preparation(df, n_test=4380, T=72, HORIZON=72):
    df = src.preprocessing.preprocess(df, 'Belgium')
    df = df.merge(series_to_supervised(df), how='right', left_index=True, right_index=True)
    train_df, test_df = train_test_split(df, n_test)
    y_scaler = MinMaxScaler()
    y_scaler.fit(train_df[['value']])
    long_scaler = MinMaxScaler()
    test_df[test_df.columns] = long_scaler.fit_transform(test_df)
    train_df[train_df.columns] = long_scaler.fit_transform(train_df)
    tensor_structure = {'X':(range(-T+1, 1), train_df.columns)}
    train_inputs = TimeSeriesTensor(train_df, 'value', HORIZON, tensor_structure)
    test_inputs = TimeSeriesTensor(test_df, 'value', HORIZON, tensor_structure)
    return train_inputs, test_inputs, y_scaler

In [22]:
def MIMO_fulldataT_preparation(df, n_test=4380, T=72, HORIZON=72):
    df = src.preprocessing.preprocess(df, 'Belgium')
    df = df.merge(series_to_supervised(df), how='right', left_index=True, right_index=True)
    df = df.merge(temp, how='left', left_index=True, right_index=True)
    train_df, test_df = train_test_split(df, n_test)
    y_scaler = MinMaxScaler()
    y_scaler.fit(train_df[['value']])
    long_scaler = MinMaxScaler()
    test_df[test_df.columns] = long_scaler.fit_transform(test_df)
    train_df[train_df.columns] = long_scaler.fit_transform(train_df)
    tensor_structure = {'X':(range(-T+1, 1), train_df.columns)}
    train_inputs = TimeSeriesTensor(train_df, 'value', HORIZON, tensor_structure)
    test_inputs = TimeSeriesTensor(test_df, 'value', HORIZON, tensor_structure)
    return train_inputs, test_inputs, y_scaler

In [38]:
temp = pd.read_csv('../data/GEP/Temp_15min.csv', index_col=0, header=0, names=['temp'])